# Deep&Cross

和之前的 Wide&Deep 网络相比，这里只是用了 Cross 网络替换掉了 Deep 部分。Cross 网络与 Deep Crossing 网络中的概念不太一样，在这里是用输入向量 $x \cdot x^T$，实现特征的交叉。

In [1]:
# build train data

import os
import numpy as np

BASEDIR = os.getcwd()

fields_dict = {}
lines = None

with open(os.path.join(BASEDIR, 'assets/datasets/criteo_ctr/small_train.txt')) as f:
    lines = f.readlines()

fields_dict = {}
for line in lines:
    line = line.strip('\n')

    for elem in line.split(' ')[1:]:
        field, feature, _ = elem.split(':')

        if field not in fields_dict:
            fields_dict[field] = {'index': len(fields_dict), 'features': {}, 'last_idx': -1}

        if feature not in fields_dict[field]['features']:
            fields_dict[field]['features'][feature] = fields_dict[field]['last_idx'] + 1
            fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1

for field in fields_dict.keys():
    if 'none' not in fields_dict[field]['features']:
        fields_dict[field]['features']['none'] = fields_dict[field]['last_idx'] + 1
        fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1


def init_field_tensor(fields_dict):
    init_tensor = np.zeros((len(fields_dict), 1))
    for field in fields_dict.keys():
        init_tensor[fields_dict[field]['index']] = fields_dict[field]['last_idx']
    return init_tensor.astype(int)


X_train = []
y_train = []

for line in lines:
    line = line.strip('\n')
    elems = line.split(' ')
    y_train.append(float(elems[0]))

    init_tensors = init_field_tensor(fields_dict)
    for elem in elems[1:]:
        field, feature, _ = elem.split(':')
        field_idx = fields_dict[field]['index']
        feature_idx = fields_dict[field]['features'][feature]
        init_tensors[field_idx] = feature_idx
    X_train.append(init_tensors)

X_train = np.concatenate(X_train, 1)
y_train = np.array(y_train)

In [12]:
# build embedding layer

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


class FieldEmbeddingBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(FieldEmbeddingBlock, self).__init__()
        self.fields_dict = fields_dict
        self.fields_embedding = {}
        self.embedding_size = embedding_size

        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            self.fields_embedding[field_idx] = nn.Embedding(len(self.fields_dict[field]['features']),
                                                            self.embedding_size)

    def forward(self, input_field_tensor):
        out = torch.zeros(input_field_tensor.shape[0] * self.embedding_size, input_field_tensor.shape[1])
        idx = 0
        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            out[idx:idx + self.embedding_size, :] = self.fields_embedding[field_idx](input_field_tensor[field_idx, :])
            idx += self.embedding_size
        return out.double()


class CrossBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(CrossBlock, self).__init__()
        self.fields_dict = fields_dict
        self.field_cnt = len(self.fields_dict)
        self.embedding_size = embedding_size
        self.layer = nn.Linear((self.field_cnt * self.embedding_size) ** 2, self.field_cnt * self.embedding_size,
                               bias=True).double()

    def forward(self, x):
        x = x.T
        origin = x
        x = torch.unsqueeze(x, 1)
        x_ = torch.transpose(x, 1, 2)
        x = x_.matmul(x)
        out = self.layer(x.reshape(origin.shape[0], -1)) + origin
        return out.T


class CrossAndWideNetwork(nn.Module):

    def __init__(self, fields_dict, embedding_size):
        super(CrossAndWideNetwork, self).__init__()
        self.fields_dict = fields_dict
        self.field_cnt = len(self.fields_dict)
        self.embedding_size = embedding_size

        self.field_embedding = FieldEmbeddingBlock(fields_dict, embedding_size)
        self.cross_block = nn.Sequential(
            CrossBlock(self.fields_dict, self.embedding_size),
            CrossBlock(self.fields_dict, self.embedding_size),
            CrossBlock(self.fields_dict, self.embedding_size)
        )

        self.deep_block = nn.Sequential(
            nn.Linear(self.embedding_size * self.field_cnt, 1024).double(),
            nn.Linear(1024, 1024, bias=True).double(),
            nn.ReLU(),
            nn.Linear(1024, 512, bias=True).double(),
            nn.ReLU(),
            nn.Linear(512, 256).double(),
            nn.ReLU(),
        )

        self.out_layer = nn.Linear(self.field_cnt * self.embedding_size + 256, 1, bias=True).double()

    def forward(self, x):
        x = self.field_embedding(x)
        out1 = self.cross_block(x)
        out2 = self.deep_block(x.T)
        out = torch.cat([out1.T, out2], 1)
        return F.sigmoid(self.out_layer(out))


In [14]:
# PyTorch Version

import torch.optim as optim

device = torch.device('cpu')
LEARNING_RATE = 1e-3

EPOCH = 100
PRINT_STEP = EPOCH / 10
N = len(y_train)

HIDDEN_DIMENSION = 8
EMBEDDING_SIZE = 8
cross_wide_network = CrossAndWideNetwork(fields_dict, EMBEDDING_SIZE)

BATCH_SIZE = 8
loss_fn = nn.BCELoss()
optimizer = optim.Adam(cross_wide_network.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCH):

    index = np.random.randint(0, X_train.shape[0], size=BATCH_SIZE)
    X_batch = torch.from_numpy(X_train[:, index]).long()
    y_batch = torch.from_numpy(y_train[index]).reshape(-1, BATCH_SIZE)

    y_hat = cross_wide_network(X_batch).reshape(-1, BATCH_SIZE)
    loss = loss_fn(y_hat, y_batch)

    loss.backward()
    optimizer.step()

    if epoch % PRINT_STEP == 0:
        print('EPOCH: %d, loss: %f' % (epoch, loss))


EPOCH: 0, loss: 0.693787
EPOCH: 10, loss: 37.500000
EPOCH: 20, loss: 37.500000
EPOCH: 30, loss: 50.000000
EPOCH: 40, loss: 37.500000
EPOCH: 50, loss: 37.500000
EPOCH: 60, loss: 50.000000
EPOCH: 70, loss: 25.000000
EPOCH: 80, loss: 50.000000
EPOCH: 90, loss: 0.000000
